# Sentinel-2 Cloud & Shadow mask with atmospheric Correction in Google Earth Engine

### Import modules and initialize Earth Engine

In [ ]:
# Atmospheric correction obtained from:
# https://github.com/samsammurphy/gee-atmcorr-S2
# Cloud and shadow mask obtained from GEE
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

import ee
import geemap
#from Py6S import * #The Py6S package is embedded into the atmospheric functions
import os, sys, time, math, datetime
from atmospheric import atmCorr, S2_L1C

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

### Test SixS from Py6S package

In [ ]:
# Instantiate
#s = SixS()

#SixS.test()

# Update geemap package
#geemap.update_package()

### Define the time

In [ ]:
# Define collection filter and cloud mask parameters
# year1 should be between 2017 to 2019 for our case of study.
year1 = 2019
year2 = year1 + 1

START_DATE = ee.Date(str(year1) + '-10-01')
END_DATE = ee.Date(str(year2) + '-01-01')
CLOUD_FILTER = 40

### Region of study

In [ ]:
# The following code will determine the region where the analysis is apply
# Rectangle with the ROI
# The images of Carmen Rosa must be obtained between the years 2017 to 2019
# between the months of October to December of those years.

# ROI Carmen Rosa
geom = ee.Geometry.Rectangle(-70.8200199999999995, -33.7983520000000013, 
                             -70.8068049999999971, -33.7804399999999987)

# ROI Quimetal
# Quimetal images should be obtained only in 2019 between the months of October to December.
#geom = ee.Geometry.Rectangle(-70.8437269,-33.6509093,
#                             -70.8383469,-33.6552673)

### Build a Sentinel-2 collection

In [ ]:
# Import and filter S2.
S2 = (ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(geom)
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
    .map(lambda x: x.clip(geom)))

### Path folder in GEE to save the imageColletion
#### GEE path to save the image inside of a image collection (GEE account)

In [ ]:
gpath = 'users/diegoalarcondiaz/Carmen_Rosa_L2A/'
#gpath = 'users/diegoalarcondiaz/Quimetal_L2A/'

### Application of Conversion from function S2_L1C
#### Conversion function in a bash process

In [ ]:
S2_L1C.conversion(geom, S2, gpath)

### Cheking the cloud and shadow mask

In [ ]:
Map = geemap.Map(center=[-70.81, -33.789], zoom=17)

vis_params = {"min": 300,
              "max": 3500,
              "bands": ["B4", "B3", "B2"]}

Map.centerObject(geom)
Map.addLayer(S2, {}, "Sentinel-2 Cloud and Shadow mask", False)
Map.add_time_slider(S2, vis_params)
Map.addLayer(geom)
Map

### Checking convertion image Sentinel-2 BOA

In [ ]:
# Create a default map
Map = geemap.Map(center=[-70.81, -33.789], zoom=17)

vis_params = {"min": 0,
              "max": 0.3,
              "bands": ["B4", "B3", "B2"]}

# Carmen Rosa L2A
S2_BOA = (ee.ImageCollection('users/diegoalarcondiaz/Carmen_Rosa_L2A')
             .filterBounds(geom)
             .filterDate(START_DATE, END_DATE)
             .map(lambda x: x.clip(geom)))

# Quimetal L2A
#S2_BOA = (ee.ImageCollection('users/diegoalarcondiaz/Quimetal_L2A')
#             .filterBounds(geom)
#             .filterDate(START_DATE, END_DATE)
#             .map(lambda x: x.clip(geom)))

Map.centerObject(geom)
Map.addLayer(S2_BOA, {}, "Sentinel-2 BOA", False)
Map.add_time_slider(S2_BOA, vis_params)
Map.addLayer(geom)
Map